In [1]:
import logging
import datetime

import numpy as np
import torch
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

import matplotlib.pyplot as plt

import config
import modules.dataloaders as dataloaders

import modules.models as cnv_model

import modules.loss as loss
import modules.metrics as metrics
import modules.train_epoch as train_epoch
import modules.val_epoch as val_epoch
import modules.utils as utils

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.22 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


# Reimport

In [2]:
# import importlib
# importlib.reload(config)

# Logger

In [3]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info(f'{config.MODEL} Classifier.\n' +  
            '\tOne Head.\n' +
            '\tWeighted for Precision.\n' +
            '\tDataset images divided by 255.\n')

# Hyperparameters Log

In [4]:
''' ============================
    Print Config Values
============================ '''
print('\nDatasets Length')
print(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
print(f'Device: {config.DEVICE}')
print('Optimizer:')
print(f'\tLearning Rate Freeze: {config.LEARNING_RATE_FREEZE}')
print(f'\tWeight Decay Freeze: {config.WEIGHT_DECAY_FREEZE}')
print(f'\tLearning Rate Fine Tuning: {config.LEARNING_RATE_FINETUNING}')
print(f'\tWeight Decay Fine Tuning: {config.WEIGHT_DECAY_FINETUNING}')
print('Scheduler:')
print(f'\tScheduler factor: {config.FACTOR}')
print(f'\tScheduler patience: {config.PATIENCE}')
print(f'\tScheduler threshold: {config.THRES}')
print(f'\tScheduler min learning rate: {config.MIN_LR}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print(f'Epochs Freeze: {config.EPOCHS_FREZZE}')
print(f'Epochs Fine Tuning: {config.EPOCHS_FINETUNING}')
print('\nIMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')

logger.info('\nDatasets Length')
logger.info(f'\tTrain and Val: {"Full" if config.DS_LEN == None else config.DS_LEN}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info('Optimizer:')
logger.info(f'\tLearning Rate Freeze: {config.LEARNING_RATE_FREEZE}')
logger.info(f'\tWeight Decay Freeze: {config.WEIGHT_DECAY_FREEZE}')
logger.info(f'\tLearning Rate Fine Tuning: {config.LEARNING_RATE_FINETUNING}')
logger.info(f'\tWeight Decay Fine Tuning: {config.WEIGHT_DECAY_FINETUNING}')
logger.info('Scheduler:')
logger.info(f'\tScheduler factor: {config.FACTOR}')
logger.info(f'\tScheduler patience: {config.PATIENCE}')
logger.info(f'\tScheduler threshold: {config.THRES}')
logger.info(f'\tScheduler min learning rate: {config.MIN_LR}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info(f'Epochs Freeze: {config.EPOCHS_FREZZE}')
logger.info(f'Epochs Fine Tuning: {config.EPOCHS_FINETUNING}')
logger.info('\nIMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')


Datasets Length
	Train and Val: 128
Device: cuda
Optimizer:
	Learning Rate Freeze: 0.001
	Weight Decay Freeze: 0.001
	Learning Rate Fine Tuning: 1e-05
	Weight Decay Fine Tuning: 0.0001
Scheduler:
	Scheduler factor: 0.8
	Scheduler patience: 2
	Scheduler threshold: 0.001
	Scheduler min learning rate: 1e-06
Batch Size: 64
Num Workers: 6
Pin Memory: True
Epochs Freeze: 2
Epochs Fine Tuning: 3

IMG DIMS:
	Width: 224
	Height: 224


# Dataloaders

In [5]:
train_loader = dataloaders.get_train_loader()
val_loader = dataloaders.get_val_loader()


TRAIN DFIRE dataset
DFire Removed wrong images: 0
DFire empty images: 56
DFire only smoke images: 37
DFire only fire images: 8
DFire smoke and fire images: 27

Train DFire dataset len: 128

TRAIN FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 58
FASDD only smoke images: 22
FASDD only fire images: 2
FASDD smoke and fire images: 46

Train FASDD UAV dataset len: 128

VAL FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 59
FASDD only smoke images: 26
FASDD only fire images: 2
FASDD smoke and fire images: 41

Val FASDD UAV dataset len: 128

TRAIN FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 53
FASDD only smoke images: 36
FASDD only fire images: 19
FASDD smoke and fire images: 20

Train FASDD CV dataset len: 128

Val FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 50
FASDD only smoke images: 32
FASDD only fire images: 20
FASDD smoke and fire images: 26

Val FASDD CV dataset len: 128

Concatenate Train DFire and

### Plot Some Train Pictures

In [6]:
for i, (img, label) in enumerate(train_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'train_pictures.png')
    #plt.show()
    plt.close()
    break

### Plot Some Val Pictures

In [7]:
for i, (img, label) in enumerate(val_loader):

    plt.subplots(8,4, figsize=(8, 16))
    
    for idx in range(config.BATCH_SIZE):
        plt.subplot(8, 4, idx+1)
        plt.imshow(img[idx].permute(1, 2, 0))
        title = ""
        if label[idx][0] == 1 and label[idx][1] == 1:
            title += "Smoke and Fire"
        elif label[idx][0] == 1 and label[idx][1] == 0:
            title += "Only Smoke"
        elif label[idx][0] == 0 and label[idx][1] == 1:
            title += "Only Fire"
        else:
            title += "Empty"
        plt.title(title)
        
        if (idx == 31):
            break
    plt.tight_layout()
    plt.savefig(config.RUN_FOLDER + 'val_pictures.png')
    plt.close()
    break

# Load Model

In [8]:
# import importlib
# importlib.reload(cnv_model)

In [9]:
model = cnv_model.setup_model(
    model_name=config.MODEL, 
    num_classes=config.NUM_CLASSES, 
    device=config.DEVICE)

Using SHUFFLE Classifier
Linear(in_features=1024, out_features=1000, bias=True)
Model Pretrained has 1024 in features in last layer


### Optimizer and Scheduler

In [10]:
optimizer = optim.Adam(model.parameters(), 
                       lr=config.LEARNING_RATE_FREEZE, 
                       weight_decay=config.WEIGHT_DECAY_FREEZE)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config.FACTOR, 
                                                 patience=config.PATIENCE, 
                                                 threshold=config.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config.MIN_LR)

### Parameters

In [11]:
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}\n')


Trainable parameters = 32866
Total parameters = 374658



### Check Model Shape: Random Input

In [12]:
dummy_input = np.random.rand(4, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)
dummy_input = torch.tensor(dummy_input, dtype=torch.float32, device=config.DEVICE)
out_test = model(dummy_input)
print(f'Model shape is {out_test}')
#print(f'BED Model Arquitecture\n{cnv_model}')

Model shape is tensor([[0.1622, 0.1120],
        [0.1670, 0.1164],
        [0.1704, 0.1148],
        [0.1600, 0.1152]], device='cuda:0', grad_fn=<AddmmBackward0>)


### Torchinfo and Print Model

In [13]:
print(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                        Output Shape              Param #
PRETRAINED_MODEL                              [1, 2]                    --
├─Sequential: 1-1                             [1, 1024, 7, 7]           --
│    └─Sequential: 2-1                        [1, 24, 112, 112]         --
│    │    └─Conv2d: 3-1                       [1, 24, 112, 112]         (648)
│    │    └─BatchNorm2d: 3-2                  [1, 24, 112, 112]         (48)
│    │    └─ReLU: 3-3                         [1, 24, 112, 112]         --
│    └─MaxPool2d: 2-2                         [1, 24, 56, 56]           --
│    └─Sequential: 2-3                        [1, 48, 28, 28]           --
│    │    └─InvertedResidual: 3-4             [1, 48, 28, 28]           (2,400)
│    │    └─InvertedResidual: 3-5             [1, 48, 28, 28]           (1,512)
│    │    └─InvertedResidual: 3-6             [1, 48, 28, 28]           (1,512)
│    │    └─InvertedResidual: 3-7             [1, 48, 28, 28]           (1,

In [14]:
logger.info("\nTorch Summary")
logger.info(summary(model, input_size=(1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W)))

In [15]:
logger.info("\nTorch Model")
logger.info(model)

# Loss Function

In [16]:
if config.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    logger.info(f'\nLoss Function: BCE')
    print(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    logger.info(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config.DEVICE, smoke_precision_weight=config.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    logger.info("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


# Loggers and Plotters for Losses and Metrics

In [17]:
train_losses_logger = utils.LogLosses()
train_metrics_logger = utils.LogMetrics()
lr_logger = utils.LogLR(log_path=config.PLOTS_FOLDER)

val_losses_logger = utils.LogLosses()
val_metrics_logger = utils.LogMetrics()

loss_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Loss')
metrics_plotter = utils.PlotMetrics(log_path=config.PLOTS_FOLDER, model_name=config.MODEL, loss_or_metric='Metric')

# Main Function to Train

In [18]:
def train_loop(model, start_epoch=0, epochs_to_train=config.EPOCHS_FREZZE):

    ''' ==============================================================
                                TRAINING LOOP
    ============================================================== '''
    start = datetime.datetime.now()
    start_time = start.strftime("%H:%M:%S")
    print(f'\n***Start Training: {start_time}\n')
    logger.info(f'\n***Start Training: {start_time}\n')
    
    # Start with infinite validation loss
    best_valid_loss = np.inf
    best_smoke_precision = 0. #torch.tensor([0.])
    smoke_f1_min_save = 0.9 #torch.tensor([0.9])
    best_mean_f1 = 0.

    if start_epoch == 0:
        epochs_plot = []
    else:
        epochs_plot = [i for i in range(start_epoch)]    

    end_epoch = start_epoch + epochs_to_train
        
    for epoch in range(start_epoch, end_epoch):

        print(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        logger.info(f'\n=== EPOCH {epoch}/{end_epoch-1} ===')
        
        #====================== TRAINING ========================#
        current_lr = train_epoch.get_lr(optimizer=optimizer)
        logger.info(f'Learning Rate = {current_lr}\n')
        lr_logger.log_lr(current_lr)
                
        train_losses, train_metrics = train_epoch.train_fn(
            loader=train_loader, 
            model=model, 
            optimizer=optimizer, 
            loss_fn=loss_fn,
            device=config.DEVICE)
        
        train_losses_logger.update_metrics(train_losses)
        train_metrics_logger.update_metrics(train_metrics)
                
        logger.info(utils.print_metrics_to_logger("TRAIN Stats", train_losses, train_metrics))
        
        #===================== VALIDATING =======================#
        with torch.no_grad():
            val_losses, val_metrics = val_epoch.eval_fn(
                loader=val_loader, 
                model=model,                         
                loss_fn=loss_fn,
                device=config.DEVICE)
            
            scheduler.step(val_losses['Total'])
            
            val_losses_logger.update_metrics(val_losses)
            val_metrics_logger.update_metrics(val_metrics)

            logger.info(utils.print_metrics_to_logger("VAL Stats", val_losses, val_metrics))
            
        epochs_plot.append(epoch)

        loss_plotter.plot_all_metrics(
            train_losses_logger.get_metrics(),
            val_losses_logger.get_metrics(),
            epochs_plot)

        metrics_plotter.plot_all_metrics(
            train_metrics_logger.get_metrics(),
            val_metrics_logger.get_metrics(),
            epochs_plot)

        lr_logger.plot_lr(epochs_plot)
        #======================= SAVING =========================#
        if ( (epoch+1) % 5 ) == 0:
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__5epoch.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            
        if best_valid_loss > val_losses['Total']:
            best_valid_loss = val_losses['Total']
            print(f"\nSaving model with new best validation loss: {best_valid_loss:.4f}")
            logger.info(f"Saving model with new best validation loss: {best_valid_loss:.4f}")
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            # save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + 'best_loss'  #+ '.onnx'
            # utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # # Save model if precision increases and F1 > 0.9
        # if ( best_smoke_precision < val_metrics['Precision'][0] ) and ( val_metrics['F1'][0] > smoke_f1_min_save ) :
        #     best_smoke_precision = val_metrics['Precision'][0]
        #     print(f"\nSaving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     logger.info(f"Saving model with new best smoke precision: {best_smoke_precision:.4f}")
        #     save_precision_name = f'best_smoke__precision={np.round(best_smoke_precision, decimals=4)}__epoch={epoch}'
        #     save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_precision_name + '.pt'
        #     utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name)  
        #     save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_precision_name #+ '.onnx'
        #     utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
            
        # Save model if precision > 0.9 and recall > 0.9
        # if ( val_metrics['Precision'][0] > 0.9 ) and ( val_metrics['Recall'][0] > 0.9 ) :
        #     print("\nSaving model with precision > 0.9 and recall > 0.9")
        #     logger.info("Saving model with precision > 0.9 and recall > 0.9")
        #     save_pre_name = f'smoke__precision={np.round(val_metrics["Precision"][0], decimals=4)}__' 
        #     save_rec_name = f'recall={np.round(val_metrics["Recall"][0], decimals=4)}__'
        #     save_pre_rec_name = save_pre_name + save_rec_name + f'epoch={epoch}'
        #     save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name + '.pt'
        #     utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
        #     save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_pre_rec_name #+ '.onnx'
        #     # utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)

        # Save model if best mean F1 increases
        val_f1_mean = (val_metrics['F1'][0] + val_metrics['F1'][1]) / 2
        if (val_f1_mean > best_mean_f1) :
            best_mean_f1 = val_f1_mean
            print(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            logger.info(f'Saving model with best Mean F1: {best_mean_f1:.4f}')
            save_f1_name = 'best_mean_F1'
            save_name = config.WEIGHTS_FOLDER + config.MODEL + '_classifier__' + save_f1_name + '.pt'
            utils.save_checkpoint(epoch, model, optimizer, scheduler, save_name) 
            # save_onnx = config.ONNX_FOLDER + config.MODEL + '_classifier__' + save_f1_name #+ '.onnx'
            # utils.export_onnx(model, (1, config.NUM_CHANNELS, config.IMG_H, config.IMG_W), save_onnx, config.DEVICE)
        
    logger.info('Saving last model')   
    torch.save(model.state_dict(), config.WEIGHTS_FOLDER + 'last_' + config.MODEL + '_classifier.pt') 
    
    #======================= FINISH =========================#
    end = datetime.datetime.now()
    end_time = end.strftime("%H:%M:%S")
    print(f'\n***Script finished: {end_time}\n')  
    print(f'Time elapsed: {end-start}')
    logger.info(f'\n***Script finished: {end_time}\n')  
    logger.info(f'Time elapsed: {end-start}')
    
    return model

In [19]:
# print(len(val_losses_logger.total))

# Training Loop

In [20]:
if __name__ == "__main__":
    
    print("Starting script\n")
    logger.info("Starting script\n")
    
    model_freeze = train_loop(model)

Starting script


***Start Training: 23:42:13


=== EPOCH 0/1 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.04it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
85.082      |40.545      |44.536      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.09it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
84.058      |40.191      |43.867      
SMOKE -> Precision: 0.4883 - Recall: 0.9947 - Accuracy: 0.4870 - F1: 0.6550
FIRE -> Precision: 0.3333 - Recall: 0.1552 - Accuracy: 0.6510 - F1: 0.2118

Saving model with new best validation loss: 84.0579
Saving model with best Mean F1: 0.4334

=== EPOCH 1/1 ===
Learning Rate = 0.001



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.33it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
81.802      |39.711      |42.090      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.13it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
80.685      |39.503      |41.183      
SMOKE -> Precision: 0.7500 - Recall: 0.3032 - Accuracy: 0.6094 - F1: 0.4318
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6979 - F1: 0.0000

Saving model with new best validation loss: 80.6854

***Script finished: 23:42:23

Time elapsed: 0:00:09.657609


# Fine Tuning

In [21]:
for param in model_freeze.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model_freeze.parameters(), 
                       lr=config.LEARNING_RATE_FINETUNING, 
                       weight_decay=config.WEIGHT_DECAY_FINETUNING)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min',
                                                 factor=config.FACTOR, 
                                                 patience=config.PATIENCE, 
                                                 threshold=config.THRES, 
                                                 threshold_mode='abs',
                                                 min_lr=config.MIN_LR)
# MODEL PARAMETERS
n_trainable = sum(p.numel() for p in model_freeze.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model_freeze.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}\n')


Trainable parameters = 374658
Total parameters = 374658



In [22]:
if __name__ == "__main__":
    
    print("Fine Tuning\n")
    logger.info("Fine Tuning\n")
    
    model = train_loop(model_freeze, start_epoch=config.EPOCHS_FREZZE, epochs_to_train=config.EPOCHS_FINETUNING)

Fine Tuning


***Start Training: 23:42:23


=== EPOCH 2/4 ===
Learning Rate = 1e-05



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.02it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
80.066      |39.340      |40.726      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.95it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
80.035      |39.410      |40.625      
SMOKE -> Precision: 0.8167 - Recall: 0.2606 - Accuracy: 0.6094 - F1: 0.3952
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6979 - F1: 0.0000

Saving model with new best validation loss: 80.0354
Saving model with best Mean F1: 0.1976

=== EPOCH 3/4 ===
Learning Rate = 1e-05



...

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
79.757      |39.369      |40.389      
SMOKE -> Precision: 0.8519 - Recall: 0.2447 - Accuracy: 0.6094 - F1: 0.3802
FIRE -> Precision: 1.0000 - Recall: 0.0086 - Accuracy: 0.7005 - F1: 0.0171

Saving model with new best validation loss: 79.7571
Saving model with best Mean F1: 0.1986

=== EPOCH 4/4 ===
Learning Rate = 1e-05



Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.12it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
79.864      |39.229      |40.636      



Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.92it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
79.557      |39.326      |40.231      
SMOKE -> Precision: 0.8421 - Recall: 0.2553 - Accuracy: 0.6120 - F1: 0.3918
FIRE -> Precision: 1.0000 - Recall: 0.0086 - Accuracy: 0.7005 - F1: 0.0171

Saving model with new best validation loss: 79.5568
Saving model with best Mean F1: 0.2045

***Script finished: 23:42:38

Time elapsed: 0:00:15.391647
